# Интерактивный анализ гиперспектральных данных с GOP

Этот Jupyter notebook демонстрирует интерактивные возможности библиотеки GOP для анализа гиперспектральных данных и оценки состояния растений.

## Содержание:
1. Загрузка и визуализация данных
2. Спектральный анализ
3. Расчет вегетационных индексов
4. Классификация состояния растений
5. Интерактивные визуализации

In [ ]:
# Импорт необходимых библиотек
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Настройка визуализации
%matplotlib inline
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")

# Добавление src в Python path
sys.path.insert(0, os.path.join(os.path.dirname('__file__'), '..', 'src'))

# Импорт модулей GOP
from src.core.pipeline import Pipeline
from src.processing.hyperspectral import HyperspectralProcessor
from src.indices.calculator import VegetationIndexCalculator
from src.segmentation.segmenter import ImageSegmenter
from src.utils.logger import setup_logger

print("Библиотеки успешно импортированы!")
print("Версия NumPy:", np.__version__)
print("Версия Matplotlib:", plt.matplotlib.__version__)

## 1. Загрузка и визуализация данных

In [ ]:
# Функция для создания примера данных
def create_sample_data(height=100, width=100, bands=50):
    """Создание примера гиперспектральных данных"""
    wavelengths = np.linspace(400, 1000, bands)
    image_data = np.zeros((height, width, bands), dtype=np.float32)
    
    for i in range(height):
        for j in range(width):
            # Создание различных спектральных сигнатур
            if (i - height//2)**2 + (j - width//2)**2 < (height//4)**2:
                # Здоровая растительность в центре
                spectrum = create_vegetation_spectrum(wavelengths, health='healthy')
            elif (i - height//4)**2 + (j - width//4)**2 < (height//8)**2:
                # Стрессовая растительность
                spectrum = create_vegetation_spectrum(wavelengths, health='stressed')
            else:
                # Почва
                spectrum = create_soil_spectrum(wavelengths)
            
            image_data[i, j, :] = spectrum + np.random.normal(0, 0.01, bands)
    
    return image_data, wavelengths

def create_vegetation_spectrum(wavelengths, health='healthy'):
    """Создание спектра растительности"""
    spectrum = np.zeros_like(wavelengths)
    
    if health == 'healthy':
        green_peak = 0.4 * np.exp(-((wavelengths - 550) / 40) ** 2)
        red_edge = 0.7 / (1 + np.exp(-(wavelengths - 720) / 15))
        nir_plateau = np.where((wavelengths >= 750) & (wavelengths <= 900), 0.8, 0)
    else:  # stressed
        green_peak = 0.2 * np.exp(-((wavelengths - 550) / 50) ** 2)
        red_edge = 0.4 / (1 + np.exp(-(wavelengths - 720) / 20))
        nir_plateau = np.where((wavelengths >= 750) & (wavelengths <= 900), 0.5, 0)
    
    spectrum = 0.1 + green_peak + red_edge + nir_plateau
    return spectrum

def create_soil_spectrum(wavelengths):
    """Создание спектра почвы"""
    return 0.15 + 0.05 * np.sin((wavelengths - 400) / 200)

# Создание примера данных
print("Создание примера гиперспектральных данных...")
image_data, wavelengths = create_sample_data(height=100, width=100, bands=50)
print(f"Размер данных: {image_data.shape}")
print(f"Спектральный диапазон: {wavelengths[0]:.1f} - {wavelengths[-1]:.1f} нм")

In [ ]:
# Визуализация данных
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

# Выбор каналов для визуализации
blue_idx = np.argmin(np.abs(wavelengths - 450))
green_idx = np.argmin(np.abs(wavelengths - 550))
red_idx = np.argmin(np.abs(wavelengths - 650))
nir_idx = np.argmin(np.abs(wavelengths - 800))

# Отдельные каналы
channels = [
    (image_data[:, :, blue_idx], 'Синий канал (450 нм)'),
    (image_data[:, :, green_idx], 'Зеленый канал (550 нм)'),
    (image_data[:, :, red_idx], 'Красный канал (650 нм)'),
    (image_data[:, :, nir_idx], 'NIR канал (800 нм)')
]

for i, (channel, title) in enumerate(channels):
    row, col = i // 3, i % 3
    im = axes[row, col].imshow(channel, cmap='viridis')
    axes[row, col].set_title(title)
    axes[row, col].axis('off')
    plt.colorbar(im, ax=axes[row, col], fraction=0.046, pad=0.04)

# RGB композит
rgb = np.stack([
    image_data[:, :, red_idx],
    image_data[:, :, green_idx],
    image_data[:, :, blue_idx]
], axis=2)

# Нормализация RGB
rgb_norm = np.zeros_like(rgb)
for i in range(3):
    band = rgb[:, :, i]
    band_min, band_max = np.percentile(band, [2, 98])
    if band_max > band_min:
        rgb_norm[:, :, i] = (band - band_min) / (band_max - band_min)

axes[1, 2].imshow(np.clip(rgb_norm, 0, 1))
axes[1, 2].set_title('RGB композит')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

## 2. Интерактивный спектральный анализ

In [ ]:
# Функция для интерактивного анализа спектров
def plot_interactive_spectra(image_data, wavelengths, x_coord=50, y_coord=50):
    """Интерактивный график спектров"""
    
    # Извлечение спектра в указанной точке
    point_spectrum = image_data[y_coord, x_coord, :]
    
    # Извлечение средних спектров для разных типов поверхностей
    center_y, center_x = image_data.shape[0] // 2, image_data.shape[1] // 2
    
    # Здоровая растительность (центр)
    vegetation_mask = np.zeros_like(image_data[:, :, 0], dtype=bool)
    for i in range(image_data.shape[0]):
        for j in range(image_data.shape[1]):
            if (i - center_y)**2 + (j - center_x)**2 < (image_data.shape[0]//4)**2:
                vegetation_mask[i, j] = True
    
    vegetation_spectrum = np.mean(image_data[vegetation_mask, :], axis=0)
    
    # Почва (внешняя область)
    soil_mask = ~vegetation_mask
    soil_spectrum = np.mean(image_data[soil_mask, :], axis=0)
    
    # Создание интерактивного графика
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Спектральные профили
    ax1.plot(wavelengths, point_spectrum, 'b-', linewidth=2, label=f'Точка ({x_coord}, {y_coord})')
    ax1.plot(wavelengths, vegetation_spectrum, 'g-', linewidth=2, label='Здоровая растительность')
    ax1.plot(wavelengths, soil_spectrum, 'brown', linewidth=2, label='Почва')
    
    ax1.set_xlabel('Длина волны (нм)')
    ax1.set_ylabel('Отражательная способность')
    ax1.set_title('Спектральные профили')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Отображение точки на изображении
    rgb_display = rgb_norm.copy()
    ax2.imshow(rgb_display)
    ax2.plot(x_coord, y_coord, 'ro', markersize=10, label=f'Точка анализа')
    ax2.set_title('Положение точки анализа')
    ax2.axis('off')
    ax2.legend()
    
    plt.tight_layout()
    plt.show()
    
    return point_spectrum, vegetation_spectrum, soil_spectrum

# Интерактивный анализ спектров
print("Интерактивный спектральный анализ")
print("Измените координаты x_coord и y_coord для анализа разных точек")

# Анализ центральной точки
point_spec, veg_spec, soil_spec = plot_interactive_spectra(
    image_data, wavelengths, x_coord=50, y_coord=50
)

In [ ]:
# Интерактивный виджет для выбора точки
try:
    from ipywidgets import interact, IntSlider
    
    def interactive_point_analysis(x_coord, y_coord):
        plot_interactive_spectra(image_data, wavelengths, x_coord, y_coord)
    
    # Создание интерактивных виджетов
    interact(interactive_point_analysis,
             x_coord=IntSlider(min=0, max=image_data.shape[1]-1, step=1, value=50, description='X:'),
             y_coord=IntSlider(min=0, max=image_data.shape[0]-1, step=1, value=50, description='Y:'))
    
except ImportError:
    print("ipywidgets не установлен. Используйте статический анализ.")
    print("Для установки выполните: pip install ipywidgets")
    
    # Анализ нескольких точек
    points = [(25, 25), (50, 50), (75, 75)]
    
    plt.figure(figsize=(12, 8))
    for x, y in points:
        spectrum = image_data[y, x, :]
        plt.plot(wavelengths, spectrum, label=f'Точка ({x}, {y})', linewidth=2)
    
    plt.xlabel('Длина волны (нм)')
    plt.ylabel('Отражательная способность')
    plt.title('Сравнение спектров в разных точках')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.show()

## 3. Расчет и анализ вегетационных индексов

In [ ]:
# Функции для расчета вегетационных индексов
def calculate_ndvi(nir, red):
    """Расчет NDVI"""
    return (nir - red) / (nir + red + 1e-8)

def calculate_gndvi(nir, green):
    """Расчет GNDVI"""
    return (nir - green) / (nir + green + 1e-8)

def calculate_ndwi(green, nir):
    """Расчет NDWI"""
    return (green - nir) / (green + nir + 1e-8)

def calculate_osavi(nir, red):
    """Расчет OSAVI"""
    return (nir - red) / (nir + red + 0.16)

# Извлечение каналов
blue_channel = image_data[:, :, blue_idx]
green_channel = image_data[:, :, green_idx]
red_channel = image_data[:, :, red_idx]
nir_channel = image_data[:, :, nir_idx]

# Расчет индексов
indices = {
    'NDVI': calculate_ndvi(nir_channel, red_channel),
    'GNDVI': calculate_gndvi(nir_channel, green_channel),
    'NDWI': calculate_ndwi(green_channel, nir_channel),
    'OSAVI': calculate_osavi(nir_channel, red_channel)
}

print("Рассчитаны вегетационные индексы:")
for name, values in indices.items():
    valid_values = values[~np.isnan(values)]
    print(f"{name}: среднее={np.mean(valid_values):.3f}, СКО={np.std(valid_values):.3f}")

In [ ]:
# Визуализация вегетационных индексов
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
axes = axes.flatten()

for i, (name, index_data) in enumerate(indices.items()):
    im = axes[i].imshow(index_data, cmap='RdYlGn', vmin=-1, vmax=1)
    axes[i].set_title(f'{name}')
    axes[i].axis('off')
    plt.colorbar(im, ax=axes[i], fraction=0.046, pad=0.04)

plt.tight_layout()
plt.show()

In [ ]:
# Интерактивный анализ распределения индексов
def plot_index_distribution(index_name='NDVI'):
    """Интерактивный график распределения индекса"""
    if index_name not in indices:
        print(f"Индекс {index_name} не найден")
        return
    
    index_data = indices[index_name]
    valid_data = index_data[~np.isnan(index_data) & (index_data > -1) & (index_data < 1)]
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
    
    # Гистограмма распределения
    ax1.hist(valid_data, bins=50, alpha=0.7, edgecolor='black', density=True)
    ax1.axvline(np.mean(valid_data), color='red', linestyle='--', linewidth=2, label=f'Среднее: {np.mean(valid_data):.3f}')
    ax1.axvline(np.median(valid_data), color='blue', linestyle='--', linewidth=2, label=f'Медиана: {np.median(valid_data):.3f}')
    ax1.set_xlabel(f'{index_name}')
    ax1.set_ylabel('Плотность')
    ax1.set_title(f'Распределение {index_name}')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Box plot
    ax2.boxplot(valid_data, vert=True, patch_artist=True)
    ax2.set_ylabel(f'{index_name}')
    ax2.set_title(f'Box plot {index_name}')
    ax2.grid(True, alpha=0.3)
    
    # Добавление статистической информации
    stats_text = f'''Статистика {index_name}:
    Количество: {len(valid_data)}
    Среднее: {np.mean(valid_data):.3f}
    СКО: {np.std(valid_data):.3f}
    Минимум: {np.min(valid_data):.3f}
    Максимум: {np.max(valid_data):.3f}
    Медиана: {np.median(valid_data):.3f}'''
    
    ax2.text(1.1, 0.5, stats_text, transform=ax2.transAxes, fontsize=10,
             verticalalignment='center', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
    plt.tight_layout()
    plt.show()

# Интерактивный выбор индекса
try:
    from ipywidgets import interact, Dropdown
    
    interact(plot_index_distribution,
             index_name=Dropdown(options=list(indices.keys()), value='NDVI', description='Индекс:'))
except ImportError:
    print("ipywidgets не установлен. Показываем статические графики.")
    
    # Статическая визуализация всех индексов
    for index_name in indices.keys():
        plot_index_distribution(index_name)

## 4. Классификация состояния растений

In [ ]:
# Функция для классификации состояния растений на основе NDVI
def classify_plant_health(ndvi_data):
    """Классификация состояния растений"""
    classification = np.zeros_like(ndvi_data, dtype=int)
    
    # Классы на основе NDVI
    classification[ndvi_data < 0.1] = 0  # Отсутствие растительности/почва
    classification[(ndvi_data >= 0.1) & (ndvi_data < 0.3)] = 1  # Бедная растительность
    classification[(ndvi_data >= 0.3) & (ndvi_data < 0.6)] = 2  # Умеренная растительность
    classification[ndvi_data >= 0.6] = 3  # Хорошея растительность
    
    return classification

# Классификация на основе NDVI
ndvi_data = indices['NDVI']
plant_classification = classify_plant_health(ndvi_data)

class_names = ['Почва/Отсутствие растительности', 'Бедная растительность', 
               'Умеренная растительность', 'Хорошая растительность']
class_colors = ['brown', 'yellow', 'lightgreen', 'darkgreen']

print("Выполнена классификация состояния растений")
print("Классы:")
for i, name in enumerate(class_names):
    count = np.sum(plant_classification == i)
    percentage = count / plant_classification.size * 100
    print(f"  {i}: {name} - {count} пикселей ({percentage:.1f}%)")

In [ ]:
# Визуализация классификации
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Карта классификации
cmap = plt.matplotlib.colors.ListedColormap(class_colors)
im1 = axes[0, 0].imshow(plant_classification, cmap=cmap, vmin=0, vmax=3)
axes[0, 0].set_title('Карта классификации состояния растений')
axes[0, 0].axis('off')

# Создание цветовой шкалы с названиями классов
cbar = plt.colorbar(im1, ax=axes[0, 0], fraction=0.046, pad=0.04)
cbar.set_ticks([0.5, 1.5, 2.5, 3.5])
cbar.set_ticklabels(class_names)

# Распределение классов
class_counts = [np.sum(plant_classification == i) for i in range(4)]
axes[0, 1].bar(class_names, class_counts, color=class_colors)
axes[0, 1].set_title('Распределение классов')
axes[0, 1].set_ylabel('Количество пикселей')
axes[0, 1].tick_params(axis='x', rotation=45)
axes[0, 1].grid(True, alpha=0.3)

# NDVI с наложением классификации
im2 = axes[1, 0].imshow(ndvi_data, cmap='RdYlGn', vmin=-1, vmax=1)
axes[1, 0].set_title('NDVI')
axes[1, 0].axis('off')
plt.colorbar(im2, ax=axes[1, 0], fraction=0.046, pad=0.04)

# Сравнение средних NDVI по классам
mean_ndvi_by_class = []
for i in range(4):
    class_mask = plant_classification == i
    if np.sum(class_mask) > 0:
        mean_ndvi_by_class.append(np.mean(ndvi_data[class_mask]))
    else:
        mean_ndvi_by_class.append(0)

axes[1, 1].bar(class_names, mean_ndvi_by_class, color=class_colors)
axes[1, 1].set_title('Средний NDVI по классам')
axes[1, 1].set_ylabel('Средний NDVI')
axes[1, 1].tick_params(axis='x', rotation=45)
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Интерактивные визуализации и анализ

In [ ]:
# Интерактивный анализ профилей
def plot_profile_analysis(profile_type='horizontal', position=50):
    """Интерактивный анализ профилей"""
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    if profile_type == 'horizontal':
        # Горизонтальный профиль
        profile_data = image_data[position, :, :]
        title = f'Горизонтальный профиль (строка {position})'
        x_axis = np.arange(image_data.shape[1])
    else:
        # Вертикальный профиль
        profile_data = image_data[:, position, :]
        title = f'Вертикальный профиль (столбец {position})'
        x_axis = np.arange(image_data.shape[0])
    
    # Профили разных каналов
    axes[0, 0].plot(x_axis, profile_data[:, blue_idx], 'b-', label='Синий (450 нм)')
    axes[0, 0].plot(x_axis, profile_data[:, green_idx], 'g-', label='Зеленый (550 нм)')
    axes[0, 0].plot(x_axis, profile_data[:, red_idx], 'r-', label='Красный (650 нм)')
    axes[0, 0].plot(x_axis, profile_data[:, nir_idx], 'y-', label='NIR (800 нм)')
    axes[0, 0].set_title(f'{title} - Спектральные каналы')
    axes[0, 0].set_xlabel('Позиция')
    axes[0, 0].set_ylabel('Отражение')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # Профили индексов
    if profile_type == 'horizontal':
        ndvi_profile = indices['NDVI'][position, :]
        gndvi_profile = indices['GNDVI'][position, :]
    else:
        ndvi_profile = indices['NDVI'][:, position]
        gndvi_profile = indices['GNDVI'][:, position]
    
    axes[0, 1].plot(x_axis, ndvi_profile, 'g-', label='NDVI', linewidth=2)
    axes[0, 1].plot(x_axis, gndvi_profile, 'b-', label='GNDVI', linewidth=2)
    axes[0, 1].set_title(f'{title} - Вегетационные индексы')
    axes[0, 1].set_xlabel('Позиция')
    axes[0, 1].set_ylabel('Значение индекса')
    axes[0, 1].legend()
    axes[0, 1].grid(True, alpha=0.3)
    axes[0, 1].axhline(y=0, color='k', linestyle='--', alpha=0.5)
    
    # Профиль классификации
    if profile_type == 'horizontal':
        class_profile = plant_classification[position, :]
    else:
        class_profile = plant_classification[:, position]
    
    axes[1, 0].plot(x_axis, class_profile, 'o-', markersize=4)
    axes[1, 0].set_title(f'{title} - Классификация')
    axes[1, 0].set_xlabel('Позиция')
    axes[1, 0].set_ylabel('Класс')
    axes[1, 0].set_yticks([0, 1, 2, 3])
    axes[1, 0].set_yticklabels([name[:10] for name in class_names])
    axes[1, 0].grid(True, alpha=0.3)
    
    # Спектральный профиль в точке максимума NDVI
    if profile_type == 'horizontal':
        max_ndvi_idx = np.nanargmax(ndvi_profile)
        spectrum_at_max = image_data[position, max_ndvi_idx, :]
    else:
        max_ndvi_idx = np.nanargmax(ndvi_profile)
        spectrum_at_max = image_data[max_ndvi_idx, position, :]
    
    axes[1, 1].plot(wavelengths, spectrum_at_max, 'b-', linewidth=2)
    axes[1, 1].set_title(f'Спектр в точке максимума NDVI')
    axes[1, 1].set_xlabel('Длина волны (нм)')
    axes[1, 1].set_ylabel('Отражение')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

# Интерактивный анализ профилей
try:
    from ipywidgets import interact, IntSlider, RadioButtons
    
    interact(plot_profile_analysis,
             profile_type=RadioButtons(options=['horizontal', 'vertical'], value='horizontal', description='Тип профиля:'),
             position=IntSlider(min=0, max=max(image_data.shape[:2])-1, step=1, value=50, description='Позиция:'))
except ImportError:
    print("ipywidgets не установлен. Показываем статические профили.")
    
    # Статический анализ горизонтального профиля
    plot_profile_analysis('horizontal', 50)

In [ ]:
# Создание интерактивной корреляционной матрицы
def create_correlation_matrix():
    """Создание и визуализация корреляционной матрицы индексов"""
    # Подготовка данных для корреляционного анализа
    index_data = []
    index_names = []
    
    for name, data in indices.items():
        valid_data = data[~np.isnan(data)].flatten()
        if len(valid_data) > 100:  # Минимальное количество точек
            index_data.append(valid_data[:1000])  # Ограничение для ускорения
            index_names.append(name)
    
    if len(index_data) < 2:
        print("Недостаточно данных для корреляционного анализа")
        return
    
    # Выравнивание данных
    min_length = min(len(data) for data in index_data)
    aligned_data = [data[:min_length] for data in index_data]
    
    # Расчет корреляционной матрицы
    correlation_matrix = np.corrcoef(aligned_data)
    
    # Визуализация
    plt.figure(figsize=(10, 8))
    sns.heatmap(correlation_matrix, 
                xticklabels=index_names, 
                yticklabels=index_names,
                annot=True, 
                cmap='coolwarm', 
                center=0,
                square=True,
                fmt='.3f')
    plt.title('Корреляционная матрица вегетационных индексов')
    plt.tight_layout()
    plt.show()
    
    # Поиск сильных корреляций
    strong_correlations = []
    for i in range(len(index_names)):
        for j in range(i + 1, len(index_names)):
            corr_value = correlation_matrix[i, j]
            if abs(corr_value) > 0.7:
                corr_type = "положительная" if corr_value > 0 else "отрицательная"
                strong_correlations.append({
                    'index1': index_names[i],
                    'index2': index_names[j],
                    'correlation': corr_value,
                    'type': corr_type
                })
    
    print("\nСильные корреляции (|r| > 0.7):")
    for corr in strong_correlations:
        print(f"  {corr['index1']} - {corr['index2']}: {corr['correlation']:.3f} ({corr['type']})")

create_correlation_matrix()

## 6. Экспорт результатов и отчеты

In [ ]:
# Функция для генерации отчета
def generate_analysis_report():
    """Генерация текстового отчета анализа"""
    report = []
    report.append("ОТЧЕТ ИНТЕРАКТИВНОГО АНАЛИЗА ГИПЕРСПЕКТРАЛЬНЫХ ДАННЫХ")
    report.append("=" * 60)
    report.append("")
    
    # Информация о данных
    report.append("1. ИНФОРМАЦИЯ О ДАННЫХ")
    report.append("-" * 30)
    report.append(f"Размер изображения: {image_data.shape[0]} x {image_data.shape[1]} пикселей")
    report.append(f"Количество спектральных каналов: {image_data.shape[2]}")
    report.append(f"Спектральный диапазон: {wavelengths[0]:.1f} - {wavelengths[-1]:.1f} нм")
    report.append("")
    
    # Статистика индексов
    report.append("2. СТАТИСТИКА ВЕГЕТАЦИОННЫХ ИНДЕКСОВ")
    report.append("-" * 30)
    for name, data in indices.items():
        valid_data = data[~np.isnan(data)]
        report.append(f"{name}:")
        report.append(f"  Среднее: {np.mean(valid_data):.4f}")
        report.append(f"  СКО: {np.std(valid_data):.4f}")
        report.append(f"  Минимум: {np.min(valid_data):.4f}")
        report.append(f"  Максимум: {np.max(valid_data):.4f}")
        report.append("")
    
    # Классификация
    report.append("3. КЛАССИФИКАЦИЯ СОСТОЯНИЯ РАСТЕНИЙ")
    report.append("-" * 30)
    for i, name in enumerate(class_names):
        count = np.sum(plant_classification == i)
        percentage = count / plant_classification.size * 100
        report.append(f"{name}: {count} пикселей ({percentage:.1f}%)")
    report.append("")
    
    # Выводы
    report.append("4. ВЫВОДЫ")
    report.append("-" * 30)
    
    mean_ndvi = np.mean(indices['NDVI'][~np.isnan(indices['NDVI'])])
    if mean_ndvi > 0.6:
        report.append("Общее состояние растительности: хорошее")
    elif mean_ndvi > 0.3:
        report.append("Общее состояние растительности: умеренное")
    else:
        report.append("Общее состояние растительности: бедное")
    
    report.append(f"Средний NDVI по всему изображению: {mean_ndvi:.4f}")
    report.append("Анализ выполнен с использованием библиотеки GOP v2.0")
    
    return "\n".join(report)

# Генерация и отображение отчета
report = generate_analysis_report()
print(report)

# Сохранение отчета в файл
output_dir = "results/interactive_analysis"
os.makedirs(output_dir, exist_ok=True)

with open(os.path.join(output_dir, "analysis_report.txt"), "w", encoding="utf-8") as f:
    f.write(report)

print(f"\nОтчет сохранен в: {output_dir}/analysis_report.txt")

In [ ]:
# Сохранение результатов в формате CSV
try:
    import pandas as pd
    
    # Создание DataFrame с результатами
    height, width = image_data.shape[:2]
    
    # Подготовка данных
    data_dict = {
        'X': np.repeat(np.arange(width), height),
        'Y': np.tile(np.arange(height), width),
        'Blue': image_data[:, :, blue_idx].flatten(),
        'Green': image_data[:, :, green_idx].flatten(),
        'Red': image_data[:, :, red_idx].flatten(),
        'NIR': image_data[:, :, nir_idx].flatten(),
        'NDVI': indices['NDVI'].flatten(),
        'GNDVI': indices['GNDVI'].flatten(),
        'NDWI': indices['NDWI'].flatten(),
        'OSAVI': indices['OSAVI'].flatten(),
        'Plant_Class': plant_classification.flatten(),
        'Class_Name': [class_names[i] for i in plant_classification.flatten()]
    }
    
    df = pd.DataFrame(data_dict)
    
    # Сохранение в CSV
    csv_path = os.path.join(output_dir, "analysis_results.csv")
    df.to_csv(csv_path, index=False)
    print(f"Результаты сохранены в CSV: {csv_path}")
    
    # Сохранение сводной статистики
    summary_stats = df.groupby('Class_Name').agg({
        'NDVI': ['mean', 'std', 'min', 'max'],
        'GNDVI': ['mean', 'std'],
        'NDWI': ['mean', 'std']
    }).round(4)
    
    excel_path = os.path.join(output_dir, "summary_statistics.xlsx")
    summary_stats.to_excel(excel_path)
    print(f"Сводная статистика сохранена в Excel: {excel_path}")
    
    # Отображение первых строк DataFrame
    print("\nПервые 5 строк результатов:")
    print(df.head())
    
except ImportError:
    print("pandas не установлен. Результаты не сохранены в CSV/Excel.")
    print("Для установки выполните: pip install pandas openpyxl")

## Заключение

В этом интерактивном notebook мы продемонстрировали:

1. **Загрузку и визуализацию гиперспектральных данных** - создание RGB композитов и анализ отдельных каналов
2. **Спектральный анализ** - интерактивное исследование спектральных профилей в разных точках
3. **Расчет вегетационных индексов** - NDVI, GNDVI, NDWI, OSAVI с визуализацией распределений
4. **Классификацию состояния растений** - автоматическая классификация на основе пороговых значений
5. **Интерактивные визуализации** - профили, корреляционные матрицы, пространственный анализ
6. **Экспорт результатов** - генерация отчетов и сохранение в различных форматах

### Возможности для дальнейшего анализа:

- Интеграция с машинным обучением для более точной классификации
- Временной анализ для мониторинга изменений растительности
- Интеграция с GPS данными для пространственного анализа
- Использование дополнительных спектральных индексов
- Применение продвинутых методов обработки изображений

Библиотека GOP предоставляет мощные инструменты для научного анализа гиперспектральных данных и может быть использована в различных исследованиях растительности и мониторинге окружающей среды.